In [ ]:
# this has to be run in a windows vm to use the windows ui accessibility features 

In [ ]:
# Gets window name
import win32gui

def enum_window_callback(hwnd, windows):
    if win32gui.IsWindowVisible(hwnd):
        window_title = win32gui.GetWindowText(hwnd)
        if "Active Trader Pro" in window_title or "Fidelity" in window_title:
            windows.append((hwnd, window_title))
    return True

windows = []
win32gui.EnumWindows(enum_window_callback, windows)
for hwnd, title in windows:
    print(f"Found window: '{title}' (HWND: {hwnd})")


In [ ]:
# scrapes window from pywinauto import Application
import time

# Connect to the specific window by HWND
app = Application(backend="uia").connect(handle=525368)
window = app.window(handle=525368)
print(f"Window title: {window.window_text()}")

# Wait for window to be ready
window.wait('visible', timeout=10)

# Print all visible text elements (e.g., quotes, positions)
for control in window.descendants():
    try:
        if control.is_visible() and control.window_text():
            print(f"Control: {control.class_name()} | Text: '{control.window_text()[:100]}...'")
    except:
        pass

# Example: Find and read a specific table or quote grid (adjust by class/name)
try:
    grid = window.child_window(class_name="DataGrid")  # Common for tables in trading apps
    print("Grid items:")
    for item in grid.children():
        print(item.window_text())
except:
    print("No DataGrid found; inspect with window.print_control_identifiers()")


In [ ]:
all_text = []
for ctrl in window.descendants():
    if ctrl.is_visible() and ctrl.window_text():
        all_text.append({
            'class': ctrl.class_name(),
            'text': ctrl.window_text()[:200],  # Truncate long entries
            'rect': ctrl.rectangle()
        })
print("\nAll visible text:")
for item in all_text:
    print(item)


In [ ]:
from pywinauto import Application, mouse
import time

HWND = 525368  # your Fidelity Trader+ window handle

app = Application(backend="uia").connect(handle=HWND)
win = app.window(handle=HWND)
win.set_focus()

# Try to locate the vertical ScrollBar
vbar = win.child_window(control_type="ScrollBar", title="Vertical")

print("ScrollBar found:", vbar.exists())


In [ ]:
from pywinauto import Application
import time

# Setup
HWND = 525368
app = Application(backend="uia").connect(handle=HWND)
win = app.window(handle=HWND)
win.set_focus()
time.sleep(1)

table_data = []
seen_rows = set()

def capture_visible_table():
    """Capture ALL Level 2 ladder controls - WIDER area"""
    new_rows = 0
    all_candidates = []  # Debug: show everything found
    
    for ctrl in win.descendants():
        try:
            if ctrl.class_name() not in ['TextBlock', 'Button']:
                continue
            txt = ctrl.window_text().strip()
            if not txt or len(txt) > 20:
                continue
            r = ctrl.rectangle()
            
            # BROADER ladder area: x=380-600, y=100-700 (covers full Level 2)
            if 380 <= r.left <= 600 and 100 <= r.top <= 700:
                all_candidates.append({
                    'top': r.top, 'left': r.left, 'text': txt, 
                    'class': ctrl.class_name(), 'rect': r
                })
                
                key = (txt, r.top, r.left)
                if key not in seen_rows:
                    seen_rows.add(key)
                    table_data.append({
                        'top': r.top, 'left': r.left, 'text': txt,
                        'class': ctrl.class_name(), 'rect': r
                    })
                    new_rows += 1
        except:
            pass
    
    print(f"\nDEBUG: Found {len(all_candidates)} total candidates in ladder area:")
    for cand in sorted(all_candidates, key=lambda x: x['top'])[:20]:  # Show top 20
        print(f"  {cand['top']:4d},{cand['left']:4d} | {cand['text']:12} | {cand['class']}")
    
    return new_rows, len(all_candidates)

# Single capture with debug
print("=== DEBUG FULL LADDER CAPTURE ===")
snapshot_count = 0
new_rows, total_cands = capture_visible_table()
snapshot_count += 1

print(f"\n=== RESULTS ===")
print(f"New rows captured: {new_rows}")
print(f"Total candidates found: {total_cands}")
print(f"Total unique in table_data: {len(table_data)}")

# Sort and show ALL captured
table_data.sort(key=lambda x: x['top'])
print(f"\nALL CAPTURED ROWS ({len(table_data)}):")
print("Top | Left | Text          | Class")
print("-" * 50)
for row in table_data:
    print(f"{row['top']:4d} | {row['left']:4d} | {row['text']:12} | {row['class']}")

print(f"\nRaw data ready in 'table_data' list")
